# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd


In [2]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Было установлено, что пропущенные значения присутствуют в двух столбцах: 'days_employed' (общий трудовой стаж в днях) и ‘total_income’ (ежемесячный доход). При этом количество пропущенных значений в обоих столбцах совпадает.Если в обоих столбцах данные не заполнены в одинаковом количестве, значит, вероятнее всего, это данные одних и тех же людей. 
Вызвав метод info() для получении информации о таблице, выяснилось, что в ней 21525 строк, пропусков в вышеуказанных столбцах 2174, что составляет 10,099883%, это довольно существенно. Столбцы с пропусками имеют тип float64, это числовой тип данных. Вполне вероятно, что эти 10% с небольшим от всех клиентов банка, не имеют официального стажа и официального дохода - поэтому они свои данные и не внесли. Поэтому имеет смысл заполнить пропуски медианным значением.

### Шаг 2.1 Заполнение пропусков

In [3]:
data_transformed = data.groupby('income_type')['total_income'].transform('median')
data['total_income'] = data['total_income'].fillna(data_transformed)
print(data['total_income'])
 

0        253875.639453
1        112080.014102
2        145885.952297
3        267628.550329
4        158616.077870
             ...      
21520    224791.862382
21521    155999.806512
21522     89672.561153
21523    244093.050500
21524     82047.418899
Name: total_income, Length: 21525, dtype: float64


In [4]:
data['days_employed'] = data['days_employed'].abs()
data_transformed = data.groupby('income_type')['days_employed'].transform('median')
data['days_employed'] = data['days_employed'].fillna(data_transformed)
print(data['days_employed'])


0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64


### Шаг 2.2 Проверка данных на аномалии и исправления.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [6]:
data['children'] = data['children'].replace(-1,1) #избавляемся от отрицательных значений в столбце с детьми
data['children'] = data['children'].replace(20,2) #также у нас есть незначительный процент семей аж с 20-ю детьми! 
#Спишем это на опечатку и заменим на 2
print(data['children'])

0        1
1        1
2        0
3        3
4        0
        ..
21520    1
21521    0
21522    1
21523    3
21524    2
Name: children, Length: 21525, dtype: int64


### Шаг 2.3. Изменение типов данных.

In [7]:
data ['debt'] = data ['debt'].astype('bool') #меняем тип столбца debt с int64 на bool
data ['total_income'] = data ['total_income'].astype('int64') #меняем тип столбца total_income с float64 на int64
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  bool   
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: bool(1), float64(1), int64(5), object(5)
memory usage: 1.8+ MB


### Шаг 2.4. Удаление дубликатов.

In [8]:
data.duplicated().sum() #считаем строки с дубликатами. Их 54
duplicated_data = data[data.duplicated()].head() #получили датафрейм с дубликатами
print(duplicated_data) 

      children  days_employed  dob_years education  education_id  \
2849         0    1574.202821         41   среднее             1   
4182         1    1574.202821         34    ВЫСШЕЕ             0   
4851         0  365213.306266         60   среднее             1   
5557         0  365213.306266         58   среднее             1   
7808         0  365213.306266         57   среднее             1   

         family_status  family_status_id gender income_type   debt  \
2849   женат / замужем                 0      F   сотрудник  False   
4182  гражданский брак                 1      F   сотрудник  False   
4851  гражданский брак                 1      F   пенсионер  False   
5557  гражданский брак                 1      F   пенсионер  False   
7808  гражданский брак                 1      F   пенсионер  False   

      total_income                  purpose  
2849        142594  покупка жилья для семьи  
4182        142594                  свадьба  
4851        118514              

Из выведенных на экран строк датафрейма заметно, что в столбце education записаны значения с разным регистром. Нужно привести их к нижнему регистру. 

In [9]:
data['education'] = data['education'].str.lower() #приводим данные к нижнему регистру

print(data.head(10))

   children  days_employed  dob_years education  education_id  \
0         1    8437.673028         42    высшее             0   
1         1    4024.803754         36   среднее             1   
2         0    5623.422610         33   среднее             1   
3         3    4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0     926.185831         27    высшее             0   
6         0    2879.202052         43    высшее             0   
7         0     152.779569         50   среднее             1   
8         2    6929.865299         35    высшее             0   
9         0    2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type   debt  total_income  \
0   женат / замужем                 0      F   сотрудник  False        253875   
1   женат / замужем                 0      F   сотрудник  False        112080   
2   женат / замужем                 0    

In [10]:
data = data.drop_duplicates().reset_index() #удаляем явные дубликаты, сбрасываем индексы строк
print(data.head(10))


   index  children  days_employed  dob_years education  education_id  \
0      0         1    8437.673028         42    высшее             0   
1      1         1    4024.803754         36   среднее             1   
2      2         0    5623.422610         33   среднее             1   
3      3         3    4124.747207         32   среднее             1   
4      4         0  340266.072047         53   среднее             1   
5      5         0     926.185831         27    высшее             0   
6      6         0    2879.202052         43    высшее             0   
7      7         0     152.779569         50   среднее             1   
8      8         2    6929.865299         35    высшее             0   
9      9         0    2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type   debt  total_income  \
0   женат / замужем                 0      F   сотрудник  False        253875   
1   женат / замужем                 0      F 

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Далее нам нужно немного облегчить вес основного датафрейма. Объявим новые датафреймы, в которой сохраним заполненные столбцы 'education', 'education_id' и 'family_status', 'family_status_id'. Для анализа нам вполне достаточно оставить только столбцы с идентификаторами. 

In [11]:
education_id = data[['education', 'education_id']] #объединяем в один датафрейм столбцы с образованием
family_status_id = data[['family_status', 'family_status_id']] #объединяем в один датафрейм столбцы с семейным статусом 
family_status_id = family_status_id.drop_duplicates().reset_index()

In [12]:
data = data.drop(columns=['education']) #удаляем из исходного датафрейма столбцы education и family_status, 
data = data.drop(columns=['family_status']) #они сохранены у нас в отдельных датафреймах выше
display(data.head(10)) #решила, что дальше нужно выводить таблицы красивее:)

,index,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,8437.673028,42,0,0,F,сотрудник,False,253875,покупка жилья
1,1,1,4024.803754,36,1,0,F,сотрудник,False,112080,приобретение автомобиля
2,2,0,5623.422610,33,1,0,M,сотрудник,False,145885,покупка жилья
3,3,3,4124.747207,32,1,0,M,сотрудник,False,267628,дополнительное образование
4,4,0,340266.072047,53,1,1,F,пенсионер,False,158616,сыграть свадьбу
5,5,0,926.185831,27,0,1,M,компаньон,False,255763,покупка жилья
6,6,0,2879.202052,43,0,0,F,компаньон,False,240525,операции с жильем
7,7,0,152.779569,50,1,0,M,сотрудник,False,135823,образование
8,8,2,6929.865299,35,0,1,F,сотрудник,False,95856,на проведение свадьбы
9,9,0,2188.756445,41,1,0,M,сотрудник,False,144425,покупка жилья для семьи


### Шаг 2.6. Категоризация дохода.

In [13]:
def total_income_category(row): #создаем функцию для категоризации по доходу
    total_income = row['total_income']
    
    if total_income <= 30000:
        return 'E'

    if total_income <= 50000:
        return 'D'

    if total_income <= 200000:
        return 'C'
    
    if total_income <= 500000:
        return 'B'
    
    return 'A'

row_values = [2500] #проверяем работу функции, для примера возьмем клиента с доходом в 2500
row_columns = ['total_income']
row = pd.Series(data=row_values, index=row_columns) 
total_income_category(row)    


'E'

Функция работает корректно. Теперь нужно создать новый столбец в датафрейме.

In [14]:
data ['total_income_category']  = data.apply(total_income_category, axis=1) #добавляем новый столбец в датафрейм
display(data.head(10))

,index,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,0,1,8437.673028,42,0,0,F,сотрудник,False,253875,покупка жилья,B
1,1,1,4024.803754,36,1,0,F,сотрудник,False,112080,приобретение автомобиля,C
2,2,0,5623.422610,33,1,0,M,сотрудник,False,145885,покупка жилья,C
3,3,3,4124.747207,32,1,0,M,сотрудник,False,267628,дополнительное образование,B
4,4,0,340266.072047,53,1,1,F,пенсионер,False,158616,сыграть свадьбу,C
5,5,0,926.185831,27,0,1,M,компаньон,False,255763,покупка жилья,B
6,6,0,2879.202052,43,0,0,F,компаньон,False,240525,операции с жильем,B
7,7,0,152.779569,50,1,0,M,сотрудник,False,135823,образование,C
8,8,2,6929.865299,35,0,1,F,сотрудник,False,95856,на проведение свадьбы,C
9,9,0,2188.756445,41,1,0,M,сотрудник,False,144425,покупка жилья для семьи,C


In [15]:
data ['total_income_category'].value_counts()

C    16015
B     4845
D      350
A      222
E       22
Name: total_income_category, dtype: int64

### Шаг 2.7. Категоризация целей кредита.

In [16]:
def purpose_category(row): #теперь создаем функцию для категоризации по цели займа.
    purpose = row['purpose']
    
    if ('автомобил') in purpose:
        return 'операции с автомобилем'

    if ('жи') in purpose:
        return 'операции с недвижимостью'

    if ('свадьб') in purpose:
        return 'проведение свадьбы'
      
    return 'получение образования'

row_values = ['покупка жилья'] #проверяем работу функции, для примера возьмем клиента с кредитом на покупку жилья
row_columns = ['purpose']
row = pd.Series(data=row_values, index=row_columns) 
purpose_category(row)    


'операции с недвижимостью'

Функция работает корректно. Теперь нужно создать новый столбец в датафрейме.

In [17]:
data ['purpose_category'] = data.apply(purpose_category, axis=1) #добавляем новый столбец в датафрейм
display(data.head(10))

,index,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,0,1,8437.673028,42,0,0,F,сотрудник,False,253875,покупка жилья,B,операции с недвижимостью
1,1,1,4024.803754,36,1,0,F,сотрудник,False,112080,приобретение автомобиля,C,операции с автомобилем
2,2,0,5623.422610,33,1,0,M,сотрудник,False,145885,покупка жилья,C,операции с недвижимостью
3,3,3,4124.747207,32,1,0,M,сотрудник,False,267628,дополнительное образование,B,получение образования
4,4,0,340266.072047,53,1,1,F,пенсионер,False,158616,сыграть свадьбу,C,проведение свадьбы
5,5,0,926.185831,27,0,1,M,компаньон,False,255763,покупка жилья,B,операции с недвижимостью
6,6,0,2879.202052,43,0,0,F,компаньон,False,240525,операции с жильем,B,операции с недвижимостью
7,7,0,152.779569,50,1,0,M,сотрудник,False,135823,образование,C,получение образования
8,8,2,6929.865299,35,0,1,F,сотрудник,False,95856,на проведение свадьбы,C,проведение свадьбы
9,9,0,2188.756445,41,1,0,M,сотрудник,False,144425,покупка жилья для семьи,C,операции с недвижимостью


Чтобы было легче проанализировать данные, сделаем сводные таблицы.
1. Есть ли зависимость между количеством детей и возвратом кредита в срок? 
Чтобы ответить на этот вопрос, необходимо сформировать сводную таблицу из следующих столбцов: 'children'(количество детей в семье), 'debt'(имел ли задолженность по возврату кредитов), и посчитать процент возврата.

In [18]:
children_debt = data.pivot_table(index=['children'], columns='debt', values='income_type', aggfunc='count')
children_debt ['percent'] = children_debt [False]/(children_debt [False]+children_debt [True])*100 #создаем столбец ‘percent’
children_debt.sort_values(by='percent').head() #выводим на экран

debt,False,True,percent
children,,,
4,37.0,4.0,90.243902
2,1926.0,202.0,90.507519
1,4410.0,445.0,90.834192
3,303.0,27.0,91.818182
0,13028.0,1063.0,92.456178


Исходя из получившейся у нас таблички мы видим, что процент возврата денег самый высокий у бездетных семей. Чуть пониже у семей с 3-мя детьми. В остальных семьях ситуация немного другая. Хуже всего обстоят дела в семьях с 3+ детей.

2. Есть ли зависимость между семейным положением и возвратом кредита в срок? Чтобы ответить на этот вопрос, нам также потребуется сводная табличка, состоящая из столбцов 'family_status_id' и 'debt'. Тут не все так просто, тк по условиям задания, нужный столбец family_status был удален, по идентификатору довольно сложно понять, о каком семейном статусе идет речь.

In [19]:
family_pivot = data.pivot_table(index=['family_status_id'], columns='debt', values='income_type', aggfunc='count')
family_debt = family_status_id.merge(family_pivot, on='family_status_id', how='left') #нам нужен не идентификатор, а его расшифровка
family_debt = family_debt.drop(columns = 'family_status_id')
family_debt ['percent'] = family_debt [False]/ (family_debt [False]+family_debt [True])*100 #создаем столбец ‘percent’
family_debt.sort_values(by='percent').head() #выводим на экран


,index,family_status,False,True,percent
4,24,Не женат / не замужем,2536,274,90.249110
1,4,гражданский брак,3763,388,90.652855
0,0,женат / замужем,11408,931,92.454818
3,19,в разводе,1110,85,92.887029
2,18,вдовец / вдова,896,63,93.430657


Получившаяся табличка ясно показывает, что те, кто потерял в свое время супруга или супругу по причине смерти последнего, являются самыми надежными заемщиками, самый высокий процент возврата именно у них.

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок? Такой вопрос, конечно, тоже может возникнуть, нам также потребуется сводная табличка, состоящая из столбца 'total_income_category', который я добавляла в датафрейм выше и столбца и 'debt'.

In [20]:
income_debt = data.pivot_table(index=['total_income_category'], columns='debt', values='income_type', aggfunc='count')
income_debt ['percent'] = income_debt [False]/(income_debt [False]+income_debt [True])*100 #создаем столбец ‘percent’
income_debt.sort_values(by='percent') #выводим на экран

debt,False,True,percent
total_income_category,,,
E,20,2,90.909091
C,14655,1360,91.507961
B,4501,344,92.899897
A,208,14,93.693694
D,329,21,94.000000


А тут получается, что самый низкий процент возврата у людей с самым низким доходом, ниже 30000, видимо с таким низким уровнем дохода незапланированные траты сильно выбивают их из бюджета. 
Лучше всех платят заемщики с доходом, чуть выше самого низкого видимо потому что им приходится тщательнее планировать свои расходы.

И последний вопрос: 4.Как разные цели кредита влияют на его возврат в срок? 
Вполне возможно, что тут тоже есть какая-то закономерность. Составим сводную табличку из столбца 'purpose_category' и столбца 'debt':

In [21]:
purpose_debt = data.pivot_table(index=['purpose_category'], columns='debt', values='income_type', aggfunc='count')
purpose_debt ['percent'] = purpose_debt [False]/(purpose_debt [False]+purpose_debt [True])*100 #создаем столбец ‘percent’
purpose_debt.sort_values(by='percent').head() #выводим на экран

debt,False,True,percent
purpose_category,,,
операции с автомобилем,3903,403,90.640966
получение образования,3643,370,90.779965
проведение свадьбы,2138,186,91.996558
операции с недвижимостью,10029,782,92.766627


Ну а в этой табличке мы видим, что самые надеждные заемщики - это люди, взявшие ипотеку. А люди, купившие автомобиль в кредит, демонстрируют самый низкий процент возврата.

# Общий вывод:

На основе проведенного исследования можно сделать вывод, что самые надежные заемщики - это не всегда семейные люди, как обычно полагают.
Высокие доходы заемщиков также не гарантируют регулярных платежей по кредиту. Самые ответственные люди - это те, которым приходится тщательно планировать свои расходы ввиду того, что они небольшие, кто по каким-то причинам потерял супруга, у кого нет детей и кто решился на такой ответственный шаг, как покупка квартиры.
Ну а самыми ненадежными заемщиками, как оказалось, являются семейные люди, с тремя детьми и предельно низким доходом, на который сильно влияют незапланированные траты, возникающие у каждого, и которые берут кредит для покупки автомобиля. 